# EVALUATION EMOTION DETECTION

In [76]:
from datasets import load_dataset



dataset = load_dataset("sem_eval_2018_task_1", 'subtask5.english')
dataset
#https://huggingface.co/datasets/sem_eval_2018_task_1

Reusing dataset sem_eval2018_task1 (C:\Users\Fossati\.cache\huggingface\datasets\sem_eval2018_task1\subtask5.english\1.1.0\c8af6e4accd23f95ac75d14477b0678a4f59d5da34e480ad7de8112ffab04a3d)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 6838
    })
    test: Dataset({
        features: ['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 3259
    })
    validation: Dataset({
        features: ['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 886
    })
})

In [77]:
import pandas as pd
sem_eval_train=pd.DataFrame(list(zip(dataset['train']['Tweet'],dataset['train']['anger'],dataset['train']['surprise'],dataset['train']['fear'],dataset['train']['joy'],dataset['train']['sadness'])), columns=['Sentence','Anger','Surprise','Fear','Joy','Sadness'])
sem_eval_train

#sfruttiamo il dataset di train per la evaluatione dal momento che il test ha problemi nell'importazione con la libreria
#non cambia nulla, anzi, abbiamo + dati, quindi anche meglio

,Sentence,Anger,Surprise,Fear,Joy,Sadness
0,“Worry is a down payment on a problem you may ...,False,False,False,False,False
1,Whatever you decide to do make sure it makes y...,False,False,False,True,False
2,@Max_Kellerman it also helps that the majorit...,True,False,False,True,False
3,Accept the challenges so that you can literall...,False,False,False,True,False
4,My roommate: it's okay that we can't spell bec...,True,False,False,False,False
...,...,...,...,...,...,...
6833,@nicky57672 Hi! We are working towards your hi...,False,False,False,False,False
6834,@andreamitchell said @berniesanders not only d...,False,True,False,False,False
6835,@isthataspider @dhodgs i will fight this guy! ...,True,False,False,False,False
6836,i wonder how a guy can broke his penis while h...,False,True,False,False,False


In [78]:
#questo dataset è per multi label classification, per cui dobbiamo rimaneggiarlo, scegliendo una delle emozioni annotate in ogni frase
#cerchiamo di equilibrare il dataset
emo_class=[]
for index,row in sem_eval_train.iterrows():

    if str(row['Surprise']) == 'True':
        emo_class.append('Surprise')
    elif str(row['Fear']) == 'True':
        emo_class.append('Fear')
    elif str(row['Sadness']) == 'True':
        emo_class.append('Sadness')
    elif str(row['Joy']) == 'True':
        emo_class.append('Joy')
    elif str(row['Anger']) == 'True':
        emo_class.append('Anger')
    else:
        emo_class.append('Neutral')
        
sem_eval_test=pd.DataFrame(list(zip(sem_eval_train['Sentence'],emo_class)), columns=['Sentence','Emotions'])
sem_eval_test

,Sentence,Emotions
0,“Worry is a down payment on a problem you may ...,Neutral
1,Whatever you decide to do make sure it makes y...,Joy
2,@Max_Kellerman it also helps that the majorit...,Joy
3,Accept the challenges so that you can literall...,Joy
4,My roommate: it's okay that we can't spell bec...,Anger
...,...,...
6833,@nicky57672 Hi! We are working towards your hi...,Neutral
6834,@andreamitchell said @berniesanders not only d...,Surprise
6835,@isthataspider @dhodgs i will fight this guy! ...,Anger
6836,i wonder how a guy can broke his penis while h...,Surprise


In [81]:
sem_eval_test.groupby('Emotions').size()

Emotions
Anger       1195
Fear        1184
Joy         1977
Neutral      612
Sadness     1509
Surprise     361
dtype: int64

In [80]:
#questo test tuttavia non è troppo affidabile per i motivi antiicpati

In [ ]:
#tweet eval


#https://github.com/cardiffnlp/tweeteval/blob/main/datasets/emotion/mapping.txt

In [82]:
tweets=pd.read_csv("C:\\Users\\Fossati\\Desktop\\Tesi\\Dati\\Dati_FineTuningBERT\\Train_Test_emotion\\Evaluation_Data_emo\\tweets.txt",sep="\n",header=None)[0].tolist()
labels=pd.read_csv("C:\\Users\\Fossati\\Desktop\\Tesi\\Dati\\Dati_FineTuningBERT\\Train_Test_emotion\\Evaluation_Data_emo\\labels.txt",sep="\n",header=None)[0].tolist()
tweeteval=pd.DataFrame(list(zip(tweets, labels)),columns=['Sentence','Emotions'])
to_drop=[]
emo=[]
for index, row in tweeteval.iterrows():
    if int(row['Emotions']) == 0:
        emo.append('Anger')
    elif int(row['Emotions']) == 1:
        emo.append('Joy')
    elif int(row['Emotions']) == 3:
        emo.append('Sadness')
    else:
        emo.append('drop')
        to_drop.append(index)
tweeteval['Emotions']=emo
tweeteval=tweeteval.drop(to_drop,axis=0 ).reset_index()
tweeteval=tweeteval.drop('index',axis=1)
tweeteval

,Sentence,Emotions
0,#Deppression is real. Partners w/ #depressed p...,Sadness
1,@user Interesting choice of words... Are you c...,Anger
2,My visit to hospital for care triggered #traum...,Sadness
3,@user Welcome to #MPSVT! We are delighted to h...,Joy
4,What makes you feel #joyful?,Joy
...,...,...
1293,I need a sparkling bodysuit . No occasion. Jus...,Joy
1294,@user I've finished reading it; simply mind-bl...,Sadness
1295,shaft abrasions from panties merely shifted to...,Anger
1296,All this fake outrage. Y'all need to stop 🤣,Anger


In [83]:
tweeteval.groupby('Emotions').size()

Emotions
Anger      558
Joy        358
Sadness    382
dtype: int64

In [ ]:
#qui abbiamo la valutazione solo su la metà delle emozioni, tuttavia è più adatto al nostro scopo rispetto al primo dataset
#che viene creato per effettuare evalutation multiclass. Noi invece vogliamo predire per ogni frase una sola emozione

In [ ]:
#proviamo ad effettuare l'evaluation con alcuni modelli vincitori degli step precedenti

In [84]:
from transformers import XLMRobertaTokenizerFast, XLMRobertaForSequenceClassification 
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import torch
import numpy as np
from sklearn.metrics import confusion_matrix

In [85]:
def prediction_test(dataset, modello, tok):  #dataset fa riferimento al set di dati che vogliamo passare, modello dipende dal tipo di FT
#poi ci sarà anche la funzione per predire senza labels

    test_texts=dataset['Sentence']
    test_labels=dataset['Emotions']

    #unique_labels = set(label for label in test_labels)
    #label2id = {label: id for id, label in enumerate(unique_labels)}
    #id2label = {id: label for label, id in label2id.items()}
    unique_labels={'Sadness', 'Surprise', 'Joy', 'Neutral', 'Anger','Fear'}
    label2id={'Sadness':0, 'Surprise':1, 'Joy':2, 'Neutral':3, 'Anger':4, 'Fear':5}
    id2label={ 0:'Sadness', 1:'Surprise', 2:'Joy', 3:'Neutral', 4:'Anger', 5:'Fear'}  

#train_encodings = tok(train_texts.tolist(), truncation=True, padding=True, max_length=max_length)
    test_encodings  = tok(test_texts.tolist(), truncation=True, padding=True, max_length=128)

#train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
    test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]
    

    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

#train_dataset = MyDataset(train_encodings, train_labels_encoded)
    test_dataset = MyDataset(test_encodings, test_labels_encoded)
    
    training_args = TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1500,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps'     # evaluate during fine-tuning so that we can see progress
)
    
    trainer = Trainer(model=modello, args=training_args)  #basta avere il modello come parametro

    predicted_results=trainer.predict(test_dataset)
    
    predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
    predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
    predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

#len(predicted_labels)

    return print(classification_report(test_labels,predicted_labels)), print(confusion_matrix(test_labels,predicted_labels,labels=['Sadness', 'Surprise', 'Joy', 'Neutral', 'Anger','Fear']))

In [86]:
#questa seconda funzione aiita il modello nella classificazione, escludendo dalla scelta le classi che non ci sono nel training

In [93]:
def prediction_test_whelp(dataset, modello, tok):  #dataset fa riferimento al set di dati che vogliamo passare, modello dipende dal tipo di FT

    test_texts=dataset['Sentence']
    test_labels=dataset['Emotions']

    #unique_labels = set(label for label in test_labels)
    #label2id = {label: id for id, label in enumerate(unique_labels)}
    #id2label = {id: label for label, id in label2id.items()}
    unique_labels={'Sadness', 'Surprise', 'Joy', 'Neutral', 'Anger','Fear'}
    label2id={'Sadness':0, 'Surprise':1, 'Joy':2, 'Neutral':3, 'Anger':4, 'Fear':5}
    id2label={ 0:'Sadness', 1:'Surprise', 2:'Joy', 3:'Neutral', 4:'Anger', 5:'Fear'}  

#train_encodings = tok(train_texts.tolist(), truncation=True, padding=True, max_length=max_length)
    test_encodings  = tok(test_texts.tolist(), truncation=True, padding=True, max_length=128)

#train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
    test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]
    

    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

#train_dataset = MyDataset(train_encodings, train_labels_encoded)
    test_dataset = MyDataset(test_encodings, test_labels_encoded)
    
    training_args = TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1500,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps'     # evaluate during fine-tuning so that we can see progress
)
    
    trainer = Trainer(model=modello, args=training_args)  #basta avere il modello come parametro

    pr=trainer.predict(test_dataset)
    

    
    count=0
    predicted_labels=[]
    for el in pr[0]:
        pred=np.argmax([pr[0][count][0],pr[0][count][2],pr[0][count][4]])
        if int(pred) == 0:
            predicted_labels.append(0)
        elif int(pred) == 1:
            predicted_labels.append(2)
        else:
            predicted_labels.append(4)
        count+=1 
        
    predicted_labels2 = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

#len(predicted_labels)

    return print(classification_report(test_labels,predicted_labels2)), print(confusion_matrix(test_labels,predicted_labels2,labels=['Sadness' ,'Joy', 'Anger']))

In [88]:
def prediction_test_baseline(dataset, modello, tok, whichdata):  #dataset fa riferimento al set di dati che vogliamo passare, modello dipende dal tipo di FT
#poi ci sarà anche la funzione per predire senza labels
    #modello = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base')
    #tok = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
    test_texts=dataset['Sentence']
    test_labels=dataset['Emotions']

    if whichdata == 1: #1 per semeval
        unique_labels={'Sadness', 'Surprise', 'Joy', 'Neutral', 'Anger','Fear'}
        label2id={'Sadness':0, 'Surprise':1, 'Joy':2, 'Neutral':3, 'Anger':4, 'Fear':5}
        id2label={ 0:'Sadness', 1:'Surprise', 2:'Joy', 3:'Neutral', 4:'Anger', 5:'Fear'}
        
        lista_labels=['Sadness', 'Surprise', 'Joy', 'Neutral', 'Anger','Fear']
    
    elif whichdata == 2: #2 per tweeteval
        
        unique_labels={'Sadness', 'Joy', 'Anger'}
        label2id={'Sadness':0,  'Joy':1,  'Anger':2}
        id2label={ 0:'Sadness',  1:'Joy', 2:'Anger'}
        
        lista_labels=['Sadness', 'Joy', 'Anger']        

#train_encodings = tok(train_texts.tolist(), truncation=True, padding=True, max_length=max_length)
    test_encodings  = tok(test_texts.tolist(), truncation=True, padding=True, max_length=128)

#train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
    test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]
    

    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

#train_dataset = MyDataset(train_encodings, train_labels_encoded)
    test_dataset = MyDataset(test_encodings, test_labels_encoded)
    
    training_args = TrainingArguments( #sono sempre gli stessi alla fine
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=2500,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
)
    
    trainer = Trainer(model=modello, args=training_args)  #basta avere il modello come parametro

    predicted_results=trainer.predict(test_dataset)
 
    predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
 
    predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
   
    predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability
        
    
    
    return print(classification_report(test_labels,predicted_labels)),print(confusion_matrix(test_labels,predicted_labels,labels=lista_labels))#aggiunta dopo
    #return predicted_results

In [89]:
#baseline mbert su benchmark






tok = DistilBertTokenizerFast.from_pretrained('distilbert-base-multilingual-cased')



#prediction_test(sem_eval_test, modello, tok, ul, l2id, id2l, epoche)
modello = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=3)


prediction_test_baseline(tweeteval, modello, tok, 2)
print('\n')
print('\n')
print('\n')

modello = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=6)
prediction_test_baseline(sem_eval_test, modello, tok, 1)


#la baseline è peggiore di tutti i modelli mbert

loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/vocab.txt from cache at C:\Users\Fossati/.cache\huggingface\transformers\28e5b750bf4f39cc620367720e105de1501cf36ec4ca7029eba82c1d2cc47caf.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\5cbdf121f196be5f1016cb102b197b0c34009e1e658f513515f2eebef9f38093.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\4708

C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       Anger       0.43      1.00      0.60       558
         Joy       0.00      0.00      0.00       358
     Sadness       0.00      0.00      0.00       382

    accuracy                           0.43      1298
   macro avg       0.14      0.33      0.20      1298
weighted avg       0.18      0.43      0.26      1298

[[  0   0 382]
 [  0   0 358]
 [  0   0 558]]








loading configuration file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/config.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\cf37a9dc282a679f121734d06f003625d14cfdaf55c14358c4c0b8e7e2b89ac9.7a727bd85e40715bec919a39cdd6f0aba27a8cd488f2d4e0f512448dcd02bf0f
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_p

C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       Anger       0.00      0.00      0.00      1195
        Fear       0.18      0.09      0.12      1184
         Joy       0.29      0.92      0.44      1977
     Neutral       0.00      0.00      0.00       612
     Sadness       0.00      0.00      0.00      1509
    Surprise       0.00      0.00      0.00       361

    accuracy                           0.28      6838
   macro avg       0.08      0.17      0.09      6838
weighted avg       0.11      0.28      0.15      6838

[[   0    0 1377    0    0  132]
 [   0    0  340    0    0   21]
 [   0    0 1818    0    0  159]
 [   0    0  550    0    0   62]
 [   0    0 1092    0    0  103]
 [   0    0 1082    0    0  102]]


(None, None)

In [90]:
#baseline XLMR su benchmark


tok = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')



#prediction_test(sem_eval_test, modello, tok, ul, l2id, id2l, epoche)
modello = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=3)


prediction_test_baseline(tweeteval, modello, tok, 2)
print('\n')
print('\n')
print('\n')

modello = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=6)
prediction_test_baseline(sem_eval_test, modello, tok, 1)


#la baseline è peggiore di tutti i modelli mbert

loading file https://huggingface.co/xlm-roberta-base/resolve/main/sentencepiece.bpe.model from cache at C:\Users\Fossati/.cache\huggingface\transformers\9df9ae4442348b73950203b63d1b8ed2d18eba68921872aee0c3a9d05b9673c6.00628a9eeb8baf4080d44a0abe9fe8057893de20c7cb6e6423cddbf452f7d4d8
loading file https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\daeda8d936162ca65fe6dd158ecce1d8cb56c17d89b78ab86be1558eaef1d76a.a984cf52fc87644bd4a2165f1e07e0ac880272c1e82d648b4674907056912bd7
loading file https://huggingface.co/xlm-roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/xlm-roberta-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/xlm-roberta-base/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at C:\Users\Fossati/.

C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       Anger       0.43      1.00      0.60       558
         Joy       0.00      0.00      0.00       358
     Sadness       0.00      0.00      0.00       382

    accuracy                           0.43      1298
   macro avg       0.14      0.33      0.20      1298
weighted avg       0.18      0.43      0.26      1298

[[  0   0 382]
 [  0   0 358]
 [  0   0 558]]








loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at C:\Users\Fossati/.cache\huggingface\transformers\87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads":

              precision    recall  f1-score   support

       Anger       0.14      0.05      0.07      1195
        Fear       0.00      0.00      0.00      1184
         Joy       0.00      0.00      0.00      1977
     Neutral       0.09      0.92      0.16       612
     Sadness       0.00      0.00      0.00      1509
    Surprise       0.00      0.00      0.00       361

    accuracy                           0.09      6838
   macro avg       0.04      0.16      0.04      6838
weighted avg       0.03      0.09      0.03      6838

[[   0    0    0 1414   95    0]
 [   0    0    0  344   17    0]
 [   0    0    0 1862  115    0]
 [   0    0    0  565   47    0]
 [   0    0    0 1139   56    0]
 [   0    0    0 1100   84    0]]


C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(None, None)

In [94]:
#mBERTstrategia finale



modello = DistilBertForSequenceClassification.from_pretrained('Emotions_fin_mbert')
tok = DistilBertTokenizerFast.from_pretrained('distilbert-base-multilingual-cased')



prediction_test(sem_eval_test, modello, tok) 
print('\n')
print('\n')
print('\n')
prediction_test(tweeteval, modello, tok) 
print('\n')
print('\n')
print('\n')
prediction_test_whelp(tweeteval, modello, tok)


loading configuration file Emotions_fin_mbert\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-multilingual-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "vocab_size": 119

using `logging_steps` to initialize `eval_steps` to 1500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 1298
  Batch size = 16


              precision    recall  f1-score   support

       Anger       0.48      0.43      0.45      1195
        Fear       0.73      0.44      0.55      1184
         Joy       0.70      0.52      0.60      1977
     Neutral       0.17      0.55      0.26       612
     Sadness       0.68      0.35      0.47      1509
    Surprise       0.17      0.40      0.24       361

    accuracy                           0.45      6838
   macro avg       0.49      0.45      0.43      6838
weighted avg       0.59      0.45      0.49      6838

[[ 532  187  123  326  273   68]
 [  14  145   81   78   27   16]
 [  64  121 1023  642  100   27]
 [  36   89   68  338   54   27]
 [  52  155   75  343  517   53]
 [  79  149   84  231  115  526]]








C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
using `logging_steps` to initialize `eval_steps` to 1500
PyTorch: setting

              precision    recall  f1-score   support

       Anger       0.85      0.44      0.58       558
        Fear       0.00      0.00      0.00         0
         Joy       0.77      0.56      0.65       358
     Neutral       0.00      0.00      0.00         0
     Sadness       0.81      0.46      0.59       382
    Surprise       0.00      0.00      0.00         0

    accuracy                           0.48      1298
   macro avg       0.40      0.24      0.30      1298
weighted avg       0.82      0.48      0.60      1298

[[175  41  39  62  21  44]
 [  0   0   0   0   0   0]
 [ 13  40 201  65  23  16]
 [  0   0   0   0   0   0]
 [ 28  77  20 136 243  54]
 [  0   0   0   0   0   0]]








              precision    recall  f1-score   support

       Anger       0.77      0.75      0.76       558
         Joy       0.63      0.79      0.70       358
     Sadness       0.75      0.60      0.67       382

    accuracy                           0.72      1298
   macro avg       0.72      0.71      0.71      1298
weighted avg       0.73      0.72      0.72      1298

[[228  81  73]
 [ 20 283  55]
 [ 54  85 419]]


(None, None)

In [ ]:
#con aiuto mBERT raggiunge un buon 72% do accuracy sulle 3 classi da predire

In [ ]:
#['Sadness' ,'Joy', 'Anger']
#questo è l'ordine delle classi per leggere la confusion matrix 3x3

In [ ]:
#lista_labels=['Sadness', 'Surprise', 'Joy', 'Neutral', 'Anger','Fear']
#questo è l'ordine per leggere la confusion matrix 6x6

In [95]:
#modello fin XLM-R

modello = XLMRobertaForSequenceClassification.from_pretrained('Emotions_fin_xlmr')
tok = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')


prediction_test(sem_eval_test, modello, tok) 
print('\n')
print('\n')
print('\n')
prediction_test(tweeteval, modello, tok) 
print('\n')
print('\n')
print('\n')
prediction_test_whelp(tweeteval, modello, tok)


loading configuration file Emotions_fin_xlmr\config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_

using `logging_steps` to initialize `eval_steps` to 1500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 1298
  Batch size = 16


              precision    recall  f1-score   support

       Anger       0.51      0.44      0.47      1195
        Fear       0.72      0.43      0.54      1184
         Joy       0.72      0.62      0.66      1977
     Neutral       0.19      0.54      0.28       612
     Sadness       0.68      0.41      0.51      1509
    Surprise       0.17      0.33      0.23       361

    accuracy                           0.49      6838
   macro avg       0.50      0.46      0.45      6838
weighted avg       0.60      0.49      0.52      6838

[[ 622  150  118  304  256   59]
 [  20  119  100   73   25   24]
 [  56   94 1223  504   74   26]
 [  35   72   90  333   54   28]
 [  54  142   78  335  527   59]
 [ 134  111  100  235   89  515]]








C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Fossati\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
using `logging_steps` to initialize `eval_steps` to 1500
PyTorch: setting

              precision    recall  f1-score   support

       Anger       0.88      0.41      0.56       558
        Fear       0.00      0.00      0.00         0
         Joy       0.82      0.57      0.67       358
     Neutral       0.00      0.00      0.00         0
     Sadness       0.83      0.53      0.65       382
    Surprise       0.00      0.00      0.00         0

    accuracy                           0.49      1298
   macro avg       0.42      0.25      0.31      1298
weighted avg       0.85      0.49      0.61      1298

[[202  37  20  58  22  43]
 [  0   0   0   0   0   0]
 [  8  36 203  79  10  22]
 [  0   0   0   0   0   0]
 [ 33  79  24 140 227  55]
 [  0   0   0   0   0   0]]








              precision    recall  f1-score   support

       Anger       0.83      0.72      0.77       558
         Joy       0.69      0.86      0.77       358
     Sadness       0.77      0.74      0.75       382

    accuracy                           0.77      1298
   macro avg       0.76      0.77      0.76      1298
weighted avg       0.78      0.77      0.77      1298

[[281  49  52]
 [ 20 309  29]
 [ 63  91 404]]


(None, None)

In [ ]:
#l'ordine con cui leggere le confusion è riportato sopra

In [ ]:
#senza aiuto xlm-r è migliore di mBERT per accuracy
#anche con aiuto performa meglio di mBERT XLMR, anche se meno evidentemente


In [ ]:
#quindi il modello che sembra essere più adatto ai nostri scopi è XLM-R, in quanto generalmente migliore di mBERT